# 4.Para la ingesta de datos se han obtenido los data frames del punto anterior como CSV

Previamente se ha extraido una muestra de 100 filas de cada fichero con sus correspondientes **id_local** para que en el momento de hacer relaciones se mas breve el procesamiento

Los datos csv se han movido a la carpeta **import** que se encuentra en el directorio de trabajo de la base de datos que he creado llamada


**nosql_pl**
**C:\Users\DYLAN\.Neo4jDesktop2\Data\dbmss\dbms-ef84f6bd-32cb-4578-bf01-39d045763473\import**

In [2]:
import pandas as pd
from nbclient.jsonutil import json_clean

# Cargar los CSVs
alojamientos = pd.read_csv('data_files/data_files_depurados/alojamientos.csv')
locales = pd.read_csv('data_files/data_files_depurados/locales.csv')
terrazas = pd.read_csv('data_files/data_files_depurados/terrazas.csv')

# 1. Sample aleatorio de 100 alojamientos (independiente)
sample_alojamientos = alojamientos.sample(n=100, random_state=42)

# 2. Encontrar ids comunes entre locales y terrazas
ids_comunes = set(locales['id_local']).intersection(set(terrazas['id_local']))

# 3. Filtrar locales y terrazas que compartan esos ids
locales_filtrados = locales[locales['id_local'].isin(ids_comunes)]
terrazas_filtradas = terrazas[terrazas['id_local'].isin(ids_comunes)]

# 4. Obtener muestra coherente (por ejemplo, 100 filas) — si hay suficientes
sample_locales = locales_filtrados.sample(n=100, random_state=42)
sample_ids = sample_locales['id_local']
sample_terrazas = terrazas_filtradas[terrazas_filtradas['id_local'].isin(sample_ids)]

# Confirmamos que los id_local coinciden en ambos sample
assert set(sample_terrazas['id_local']) == set(sample_locales['id_local'])

# Resultado:
# - sample_alojamientos: muestra de 100 filas
# - sample_locales y sample_terrazas: muestra coherente con los mismos id_local

# Guardar
sample_alojamientos.to_csv('sample_alojamientos.csv', index=False)
sample_locales.to_csv('sample_locales.csv', index=False)
sample_terrazas.to_csv('sample_terrazas.csv', index=False)



```json
LOAD CSV WITH HEADERS FROM 'file:///sample_locales.csv' AS row
CREATE (:Local {
  id_local: toInteger(row.id_local),
  nombre: row.rotulo,
  distrito: row.desc_distrito_local,
  barrio: row.desc_barrio_local,
  apertura: row.hora_apertura1,
  cierre: row.hora_cierre1,
  codigo_postal: row.cod_postal,
  actividades: row.actividades
});

```json
LOAD CSV WITH HEADERS FROM 'file:///sample_terrazas.csv' AS row
WITH row
WHERE row.id_terraza IS NOT NULL
CREATE (:Terraza {
  id_terraza: toInteger(row.id_terraza),
  superficie: toFloat(row.superficie_es),
  situacion: row.desc_situacion_terraza,
  ubicacion: row.desc_ubicacion_terraza,
  mesas: toInteger(row.mesas_es),
  sillas: toInteger(row.sillas_es),
  fecha_licencia: row.fecha_confir_ult_decreto_resol
});


## Importacon de Alojamientos
```json
LOAD CSV WITH HEADERS FROM 'file:///sample_alojamientos.csv' AS row
CREATE (:Alojamiento {
  id: toInteger(row.id),
  nombre: row.name,
  anfitrion_id: toInteger(row.host_id),
  anfitrion_nombre: row.host_name,
  distrito: row.desc_distrito_local,
  barrio: row.desc_barrio_local,
  latitud: toFloat(row.latitude),
  longitud: toFloat(row.longitude),
  tipo_habitacion: row.room_type,
  precio: toInteger(row.price),
  noches_minimas: toInteger(row.minimum_nights),
  reseñas: toInteger(row.number_of_reviews),
  ultima_reseña: row.last_review,
  reseñas_mensuales: toFloat(row.reviews_per_month),
  disponibilidad: toInteger(row.availability_365),
  licencia: row.license
});


## Creacion de relacion entre TERRAZAS y LOCALES
```json
MATCH (t:Terraza), (l:Local)
WHERE t.id_local = l.id_local
MERGE (l)-[:TIENE_TERRAZA]->(t)

## Creacion de los nodos BARRIO a partir de Locales
```json
LOAD CSV WITH HEADERS FROM 'file:///sample_locales.csv' AS row
MERGE (:Barrio {nombre: row.desc_barrio_local});

## Creacion de relaciones **Ubicado_en** desde AirBnB (alojamientos -> barrio)
```json
LOAD CSV WITH HEADERS FROM 'file:///sample_alojamientos.csv' AS row
MATCH (a:Alojamiento {nombre: row.name})
MATCH (b:Barrio {nombre: row.desc_barrio_local})
MERGE (a)-[:Ubicado_en]->(b);


## Creacion de relaciones **Ubicado_en** desde Locales (Local -> Barrio)
```json
LOAD CSV WITH HEADERS FROM 'file:///sample_locales.csv' AS row
MATCH (l:Local {id_local: toInteger(row.id_local)})
MATCH (b:Barrio {nombre: row.desc_barrio_local})
MERGE (l)-[:Ubicado_en]->(b);


## Crear relaciones Ubicado_en para terrazas (Terrazas -> Barrio)
```json
LOAD CSV WITH HEADERS FROM 'file:///sample_terrazas.csv' AS row
MATCH (t:Terraza {id_local: toInteger(row.id_local)})
MATCH (l:Local {id_local: toInteger(row.id_local)})-[:Ubicado_en]->(b:Barrio)
MERGE (t)-[:Ubicado_en]->(b);


## Crear el atributo **distancia** en la relacion **Ubicado_en**
```json
MATCH ()-[r:Ubicado_en]->()
SET r.distancia = round(rand() * 0.9 + 0.1, 2);


## Crear las relaciones **Cercano_a** y el atributo **distancia** con una restriccion



## Cercano_a Local y Terraza
```json
MATCH (l:Local), (t:Terraza)
WITH l, t, round(rand(), 2) AS distancia
WHERE distancia <= 0.5
MERGE (l)-[:Cercano_a {distancia: distancia}]->(t);


## Cercano_a Local y Alojamiento
```json
MATCH (l:Local), (a:Alojamiento)
WITH l, a, round(rand(), 2) AS distancia
WHERE distancia <= 0.5
MERGE (l)-[:Cercano_a {distancia: distancia}]->(a);


## Cercano_a Terraza y Alojamiento
```json
MATCH (t:Terraza), (a:Alojamiento)
WITH t, a, round(rand(), 2) AS distancia
WHERE distancia <= 0.5
MERGE (t)-[:Cercano_a {distancia: distancia}]->(a);


## Creacion de relacion **Relacion_con** y los atributos que pide el enunciado

```json
MATCH (l:Local)
WHERE l.actividades IS NOT NULL
WITH l, apoc.convert.fromJsonList(l.actividades) AS acts
SET l.actividad_principal = acts[0].desc_epigrafe




```json
MATCH (l1:Local), (l2:Local)
WHERE l1.actividad_principal = l2.actividad_principal AND l1.nombre < l2.nombre
MERGE (l1)-[:Relacionado_con {motivo: 'Misma actividad'}]->(l2);


## QUERYS

## 2.1. Consulta todos los locales y terrazas del barrio “Salamanca”.


In [11]:
from neo4j import GraphDatabase

# Configura la conexión con tu base de datos Neo4j
uri = "bolt://localhost:7687"
user = "neo4j"
password = "100424Fer."  # Reemplaza por tu contraseña real

# Crear el driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Define la consulta Cypher
query = """
MATCH (b:Barrio {nombre: "tetuan"})<-[:Ubicado_en]-(n)
WHERE n:Local OR n:Terraza
RETURN n.nombre AS nombre, labels(n) AS tipo
"""

# Función para ejecutar la consulta
def run_query(tx, query):
    result = tx.run(query)
    return [record.data() for record in result]

# Ejecutar la consulta
with driver.session() as session:
    resultados = session.read_transaction(run_query, query)

# Mostrar los resultados
for fila in resultados:
    print(fila)

# Cerrar conexión
driver.close()



C:\Users\DYLAN\AppData\Local\Temp\ipykernel_11260\37816999.py:25: DeprecationWarning: read_transaction has been renamed to execute_read
  resultados = session.read_transaction(run_query, query)


## 2.2. Consulta los alojamientos cuyo precio supere los 100€.

In [12]:
from neo4j import GraphDatabase

# Datos de conexión
uri = "bolt://localhost:7687"
user = "neo4j"
password = "100424Fer."

# Crear driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Consulta Cypher
query = """
MATCH (a:Alojamiento)
WHERE a.precio > 100
RETURN a.nombre AS nombre, a.precio AS precio
"""

# Función para ejecutar consulta
def run_query(tx, query):
    result = tx.run(query)
    return [record.data() for record in result]

# Ejecutar y mostrar resultados
with driver.session() as session:
    resultados = session.read_transaction(run_query, query)

# Mostrar en consola
for fila in resultados:
    print(fila)

# Cerrar conexión
driver.close()


C:\Users\DYLAN\AppData\Local\Temp\ipykernel_11260\3866369696.py:25: DeprecationWarning: read_transaction has been renamed to execute_read
  resultados = session.read_transaction(run_query, query)


{'nombre': 'blueground | chamberi, doorman, nr school', 'precio': 171}
{'nombre': 'apartamento entero en pleno centro de madrid 1-2', 'precio': 139}
{'nombre': 'estudio con terraza en madrid centro', 'precio': 114}
{'nombre': 'self check in covid free the bohemian la latina - heart of madrid', 'precio': 103}
{'nombre': 'modern and bright duplex in la latina/plaza mayor', 'precio': 155}
{'nombre': 'cibeles by skykey', 'precio': 173}
{'nombre': 'piso moderno nuevo en chueca, centro de madrid.', 'precio': 174}
{'nombre': 'nuevo departamento - salamanca', 'precio': 120}
{'nombre': 'charming madrid plaza - en la plaza mayor', 'precio': 117}
{'nombre': 'new! amazing luxurious apartment in madrid center', 'precio': 205}
{'nombre': 'apartamento centro madrid malasaña m (mal27)', 'precio': 220}
{'nombre': 'tajahri - 2 bedrooms in azca', 'precio': 137}
{'nombre': '3 rooms apartment in puerta del sol', 'precio': 288}
{'nombre': 'designer 2 room flat in malasaña by mantehouse', 'precio': 168}
{'no

## 2.3. Consulta los barrios donde los alojamientos no cuentan con dormitorios.

In [14]:
from neo4j import GraphDatabase
import pandas as pd

# Datos de conexión
uri = "bolt://localhost:7687"
user = "neo4j"
password = "100424Fer."

# Crear driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Consulta Cypher
query = """
MATCH (a:Alojamiento)-[:Ubicado_en]->(b:Barrio)
WHERE a.numero_habitaciones = 1
RETURN b.nombre AS barrio, COUNT(a) AS total_alojamientos
"""

# Función para ejecutar consulta
def run_query(tx, query):
    result = tx.run(query)
    return [record.data() for record in result]

# Ejecutar y mostrar resultados
with driver.session() as session:
    resultados = session.read_transaction(run_query, query)

# Convertir a DataFrame y mostrar
df = pd.DataFrame(resultados)
print(df)

# Cerrar conexión
driver.close()


C:\Users\DYLAN\AppData\Local\Temp\ipykernel_11260\819170619.py:26: DeprecationWarning: read_transaction has been renamed to execute_read
  resultados = session.read_transaction(run_query, query)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: numero_habitaciones)} {position: line: 3, column: 9, offset: 57} for query: '\nMATCH (a:Alojamiento)-[:Ubicado_en]->(b:Barrio)\nWHERE a.numero_habitaciones = 1\nRETURN b.nombre AS barrio, COUNT(a) AS total_alojamientos\n'


Empty DataFrame
Columns: []
Index: []


## 2.4. Consulta los barrios donde los alojamientos no cuentan con reseñas.


In [15]:
from neo4j import GraphDatabase
import pandas as pd

# Conexión a Neo4j
uri = "bolt://localhost:7687"
user = "neo4j"
password = "100424Fer."

driver = GraphDatabase.driver(uri, auth=(user, password))

# Consulta Cypher
query = """
MATCH (a:Alojamiento)-[:Ubicado_en]->(b:Barrio)
WHERE a.reseñas = 0
RETURN b.nombre AS barrio, COUNT(a) AS total_alojamientos
"""

# Ejecutar la consulta
def run_query(tx, query):
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    results = session.read_transaction(run_query, query)

df = pd.DataFrame(results)
print(df)

# Cerrar la conexión
driver.close()


C:\Users\DYLAN\AppData\Local\Temp\ipykernel_11260\1958932681.py:24: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(run_query, query)


             barrio  total_alojamientos
0  puerta del angel                   1
1          justicia                   2
2         trafalgar                   2
3    cuatro caminos                   1
4     valdeacederas                   1
5       embajadores                   2
6           acacias                   1
7       universidad                   1
8         recoletos                   1
9              goya                   1


## QUERYS FINALES

In [16]:
from neo4j import GraphDatabase
import pandas as pd

# Conexión
uri = "bolt://localhost:7687"
user = "neo4j"
password = "100424Fer."

driver = GraphDatabase.driver(uri, auth=(user, password))

# Consulta Cypher
query = """
MATCH (n)-[r]->(m)
RETURN n, type(r) AS relacion, r, m
"""

# Ejecutar la consulta
def run_query(tx, query):
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    results = session.read_transaction(run_query, query)

# Convertimos los nodos a un DataFrame legible
df = pd.DataFrame([{
    'nodo_origen': record['n'],
    'relacion': record['relacion'],
    'atributo_relacion': record['r'],
    'nodo_destino': record['m']
} for record in results])

pd.set_option('display.max_colwidth', None)
print(df)

driver.close()


C:\Users\DYLAN\AppData\Local\Temp\ipykernel_11260\4186212254.py:23: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(run_query, query)


                                                                                                                                                                                                                                                                                                                                                                     nodo_origen  \
0      {'distrito': 'chamartin', 'cierre': '00:00:00', 'barrio': 'el viso', 'codigo_postal': '28036', 'id_local': 50002004, 'actividades': '[{'id_seccion': 'i', 'desc_seccion': 'hostelería', 'id_division': '56', 'desc_division': 'servicios de comidas y bebidas', 'id_epigrafe': '561001', 'desc_epigrafe': 'restaurante'}]', 'nombre': 'pomodoro', 'apertura': '00:00:00'}   
1      {'distrito': 'chamartin', 'cierre': '00:00:00', 'barrio': 'el viso', 'codigo_postal': '28036', 'id_local': 50002004, 'actividades': '[{'id_seccion': 'i', 'desc_seccion': 'hostelería', 'id_division': '56', 'desc_division': 'servicios de comidas y beb

In [17]:
query_locales = """
MATCH (n:Local)
RETURN n.nombre AS nombre, n.tipo_actividad AS tipo_actividad, n.horario AS horario, n.licencia AS licencia
"""

with driver.session() as session:
    results_locales = session.run(query_locales)
    df_locales = pd.DataFrame([record.data() for record in results_locales])

print(df_locales)


C:\Users\DYLAN\AppData\Local\Temp\ipykernel_11260\188124446.py:6: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: tipo_actividad)} {position: line: 3, column: 30, offset: 46} for query: '\nMATCH (n:Local)\nRETURN n.nombre AS nombre, n.tipo_actividad AS tipo_actividad, n.horario AS horario, n.licencia AS licencia\n'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {

                              nombre tipo_actividad horario licencia
0                           pomodoro           None    None     None
1                           el bombo           None    None     None
2                     mama framboise           None    None     None
3                          surrender           None    None     None
4              el rinconcito de juan           None    None     None
..                               ...            ...     ...      ...
95  bar restaurante rincón de zamora           None    None     None
96                             luigi           None    None     None
97                    el quinto vino           None    None     None
98                           bar eva           None    None     None
99            bar restaurante caribe           None    None     None

[100 rows x 4 columns]


In [18]:
query_relaciones = """
MATCH (n)-[r:Ubicado_en]->(m:Barrio)
RETURN n.nombre AS entidad, m.nombre AS barrio, r.distancia AS distancia
"""

with driver.session() as session:
    results_relaciones = session.run(query_relaciones)
    df_relaciones = pd.DataFrame([record.data() for record in results_relaciones])

print(df_relaciones)


                      entidad         barrio  distancia
0                    pomodoro        el viso       0.26
1                    la pausa        el viso       0.21
2                  café segre        el viso       0.39
3                         num        el viso       0.52
4                         num        el viso       0.25
..                        ...            ...        ...
261  bar paloma ii cerveceria         ventas       0.19
262                       num         ventas       0.26
263    bar restaurante caribe  bellas vistas       0.33
264                  马德里的温馨公寓  bellas vistas       0.34
265                       num  bellas vistas       0.38

[266 rows x 3 columns]


C:\Users\DYLAN\AppData\Local\Temp\ipykernel_11260\192982701.py:6: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
